In [1]:
import networkx as nx
import numpy as np
import torch_geometric
import torch

import matplotlib.pyplot as plt

from torch_geometric.data import Dataset, Data
from torch_geometric.loader.dataloader import DataLoader

from GraphDataset import RandomGraphDataset
from GraphToSequence import graphToSequence, sequenceToGraph

In [2]:
data_folder_training = "/eos/user/c/czeh/random_graph_training"
data_folder_test = "/eos/user/c/czeh/random_graph_test"

dataset_training = RandomGraphDataset(data_folder_training, nodes=100, data_count=1000)
dataset_test = RandomGraphDataset(data_folder_test, nodes=100, data_count=50)

Processing...
/eos/home-i04/c/czeh/SWAN_projects/GraphToSequence/GraphDataset.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  run = torch.load(raw_path)
Done!
Processin

In [3]:
dataset_training.get(1).edge_index.T

/eos/home-i04/c/czeh/SWAN_projects/GraphToSequence/GraphDataset.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(osp.join(self.processed_dir, f'data_{i

tensor([[ 0, 20],
        [49, 82],
        [56, 85],
        [76, 95],
        [53, 87],
        [60, 99],
        [ 9, 32],
        [19, 94],
        [61, 83],
        [17, 82],
        [74, 93],
        [34, 35],
        [ 2, 17],
        [41, 52],
        [27, 75],
        [39, 90],
        [62, 83],
        [25, 94],
        [27, 34],
        [ 9, 56],
        [ 9, 66],
        [82, 88],
        [28, 98],
        [ 5, 69],
        [48, 69],
        [21, 48],
        [60, 98],
        [35, 38],
        [42, 82],
        [71, 74],
        [67, 91],
        [23, 72],
        [26, 39],
        [ 9, 15],
        [ 5, 57],
        [31, 90],
        [ 0, 32],
        [59, 88],
        [78, 94],
        [49, 71],
        [ 2, 94],
        [18, 51],
        [70, 97],
        [ 3, 85],
        [32, 96],
        [28, 92],
        [21, 62],
        [24, 92],
        [72, 97],
        [14, 78]])

In [4]:
origG = torch_geometric.utils.to_networkx(dataset_training.get(5), to_undirected=True)
# nx.draw_circular(origG, with_labels=True)

In [5]:
seq = graphToSequence(dataset_training.get(5).edge_index.numpy())

In [6]:
res_nodes, res_edges = sequenceToGraph(seq)

data = Data(x=dataset_training.get(5).x, num_nodes=res_nodes.shape[0], edge_index=torch.from_numpy(res_edges))
G = torch_geometric.utils.to_networkx(dataset_training.get(5), to_undirected=True)
# nx.draw_circular(G, with_labels=True)



In [7]:
print(nx.graph_edit_distance(origG, G))

0.0


In [8]:
train_dl = DataLoader(dataset_training, shuffle=False)

In [9]:
i = 0
for data in train_dl:
    origG = torch_geometric.utils.to_networkx(data, to_undirected=True)
    seq = graphToSequence(data.edge_index.numpy())
    
    res_nodes, res_edges = sequenceToGraph(seq)
    newData = Data(x=data.x, num_nodes=data.num_nodes, edge_index=torch.from_numpy(res_edges))
    G = torch_geometric.utils.to_networkx(newData, to_undirected=True)
    
    if (nx.graph_edit_distance(origG, G) > 0):
        print(i)
        print(nx.graph_edit_distance(origG, G))
        print(res_edges.T)
        fig, ax = plt.subplots(1, 2)
        nx.draw_circular(origG, with_labels=True, ax=ax[0])
        nx.draw_circular(G, with_labels=True, ax=ax[1])
        break
    i += 1